In [4]:
import sys
project_path = r"D:/DaiHoc/machinelearning/TLCN/DoAnTotNghiep_chat_bot/"
sys.path.append(project_path)
from source.function.utils_result import RAG
from source.search.utils_search import Qdrant_Utils
from source.rerank.utils_rerank import Rerank_Utils  
from source.model.embedding_model import Sentences_Transformer_Embedding
from source.model.extract_model import Bert_Extract
from source.model.generate_model import Gemini
from source.model.rerank_model import Cohere
from source.data.vectordb.qdrant import Qdrant_Vector
from source.core.config import Settings
from source.generate.generate import Gemini_Generate
from source.extract.utils_extract import Extract_Information
from source.schema.chatbot_querry import ChatbotQuery
setting=Settings()
gemini=Gemini(setting)
print(setting.RERANK)
cohere=Cohere(setting)
print(setting.RERANK)
bert=Bert_Extract(setting)
sentences_transformer_embedding=Sentences_Transformer_Embedding(setting)
qdrant=Qdrant_Vector(setting,sentences_transformer_embedding)
rerank_Utils=Rerank_Utils(cohere)
extract_Utils= Extract_Information(bert)
generate_Utils=Gemini_Generate(gemini,setting)
qdrant_Utils=Qdrant_Utils(qdrant, generate_Utils)
rag=RAG(generate_Utils,extract_Utils,qdrant_Utils,rerank_Utils,setting,sentences_transformer_embedding)
import pandas as pd
from tqdm import tqdm
import google.generativeai as genai 
from sklearn.metrics.pairwise import cosine_similarity
import os

hghaan/rerank_model
hghaan/rerank_model


In [5]:
print(rerank_Utils.model_rerank)

## 1.Call API System

In [ ]:
import os
import pandas as pd
from tqdm import tqdm

# Đọc file CSV chứa câu hỏi và câu trả lời đã có (nếu có)
df = pd.read_csv('./data/data_processed/honnhan.csv')
df=df.head(50)
# Nếu chưa có cột 'answer_from_chatbot', thêm cột với giá trị None
if 'answer_from_chatbot' not in df.columns:
    df['answer_from_chatbot'] = None

def sinh_cau_tra_loi_batch(df_batch: pd.DataFrame) -> list[str]:
    """
    Sinh câu trả lời cho mỗi câu hỏi trong batch.
    Trả về danh sách answer tương ứng.
    """
    answers = []
    for question in tqdm(df_batch['question'], desc="Sinh trả lời"):    
        article_doc, _ = rag.get_Article_Content_Results(question)
        answers.append(article_doc)
    return answers

# Chỉ lấy các dòng chưa có câu trả lời (NaN hoặc rỗng)
df_unanswered = df[df['answer_from_chatbot'].isna() | (df['answer_from_chatbot'].str.strip() == '')]

print("==== Bắt đầu sinh trả lời cho các dòng bị thiếu ====")

# Sinh câu trả lời cho toàn bộ các dòng thiếu
df_unanswered['answer_from_chatbot'] = sinh_cau_tra_loi_batch(df_unanswered)

# Cập nhật lại vào df gốc
df.loc[df_unanswered.index, 'answer_from_chatbot'] = df_unanswered['answer_from_chatbot']

# Lưu kết quả cuối cùng
final_output_path = '../Eval_System/data/data_processed/final_response_honnhan_api.csv'
df.to_csv(final_output_path, index=False)
print(f"✅ Đã cập nhật và lưu toàn bộ vào: {final_output_path}")


==== Bắt đầu sinh trả lời cho các dòng bị thiếu ====


Sinh trả lời:   0%|          | 0/50 [00:00<?, ?it/s]

### 2.Eval System

> Configure

In [ ]:
setting=Settings()

In [ ]:
genai.configure(api_key=gemini.key_manager.get_next_key())

> Get embedding

In [ ]:
def get_gemini_embedding(text: str,
                         model_name: str = "models/text-embedding-004") -> list[float]:
    res = genai.embed_content(
        model=model_name,
        content=text
    )
    return res["embedding"]

In [ ]:
df = pd.read_csv('./data/data_processed/final_response_honnhan_api.csv')
# Lấy embedding cho Answer và answer_from_chatbot
df['answer_embedding'] = df['answer'].apply(get_gemini_embedding)
df['answer_from_chatbot_embedding'] = df['answer_from_chatbot'].apply(get_gemini_embedding)

# Tính cosine similarity giữa Answer và answer_from_chatbot
df['cosine_similarity'] = df.apply(
    lambda row: cosine_similarity(
        [row['answer_embedding']],
        [row['answer_from_chatbot_embedding']]
    )[0][0],
    axis=1
)
# Tính trung bình của cosine similarity
average_cosine_similarity = df['cosine_similarity'].mean()

print(f"Trung bình cosine similarity giữa Answer và answer_from_chatbot: {average_cosine_similarity:.4f}")